In [1]:
import math
import torchtext
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter
from torchtext.vocab import Vocab
import io
import time
import pandas as pd
import numpy as np
import pickle
import sentencepiece as spm
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [2]:
be_path = "/kaggle/input/bengali-to-english-datasets/Bangla_text.txt"

In [3]:
def read_bdata(be_path):
    with open(be_path, "r") as b_file:
        data = b_file.readlines()
    return data

In [4]:
be_data = read_bdata(be_path)

In [5]:
be_data[:10]

['আবার সবাইকে যার যার ইচ্ছামতো চলতে দিলে সমতা রক্ষা করা অসম্ভব হয়ে যায়।\n',
 'তুরুপ\n',
 'পদদলিত করা\n',
 '"সেগুলো হৃদয়কে উষ্ণ করে এবং রোজকার বোঝাগুলোকে হালকা করে।\n',
 'আমি ভালোবাসি তোমাকে ।\n',
 'পোর্ট কোম্পানি লিমিটেড - কেপিসিএল\n',
 'তোলপাড়\n',
 'এছাড়াও ক্লেমঁসো উইলসনের ১৪ দফার ব্যাপারে সংশয়ী এবং হতাশ ছিলেন। তিনি অভিযোগ করে বলেন, "মিস্টার উইলসনের ১৪ দফা বিরক্তিকর।\n',
 'আল-জাজিরার সঙ্গে জালুদের সাক্ষাতকার এবং তার দৃষ্টিভঙ্গীর বিষয়ে অন্যান্য টুইটার ব্যবহারকারীরা মন্তব্য করেন:\n',
 'ক্লাব.\n']

In [6]:
bdata=[string.replace('\n', '') for string in be_data]

In [7]:
bdata[:10]

['আবার সবাইকে যার যার ইচ্ছামতো চলতে দিলে সমতা রক্ষা করা অসম্ভব হয়ে যায়।',
 'তুরুপ',
 'পদদলিত করা',
 '"সেগুলো হৃদয়কে উষ্ণ করে এবং রোজকার বোঝাগুলোকে হালকা করে।',
 'আমি ভালোবাসি তোমাকে ।',
 'পোর্ট কোম্পানি লিমিটেড - কেপিসিএল',
 'তোলপাড়',
 'এছাড়াও ক্লেমঁসো উইলসনের ১৪ দফার ব্যাপারে সংশয়ী এবং হতাশ ছিলেন। তিনি অভিযোগ করে বলেন, "মিস্টার উইলসনের ১৪ দফা বিরক্তিকর।',
 'আল-জাজিরার সঙ্গে জালুদের সাক্ষাতকার এবং তার দৃষ্টিভঙ্গীর বিষয়ে অন্যান্য টুইটার ব্যবহারকারীরা মন্তব্য করেন:',
 'ক্লাব.']

In [8]:
en_path = "/kaggle/input/bengali-to-english-datasets/English_text.txt"

def read_edata(en_path):
    with open(en_path, "r") as e_file:
        data = e_file.readlines()
    return data

en_data = read_edata(en_path)
edata=[string.replace('\n', '') for string in en_data]

In [9]:
edata[:10]

['Guaranteeing that every individual will be free to do as he wishes inevitably short-changes equality.',
 'trump',
 'overrides',
 '"They warm the heart and ease the daily load.',
 'I love you.',
 'Port Company Limited - KPCL',
 'commotions',
 'Clemenceau also expressed skepticism and frustration with Wilson\'s Fourteen Points: "Mr. Wilson bores me with his fourteen points", complained Clemenceau.',
 "Other Twitter users went on commenting on Jalloud's interview for Al-Jazeera and his attitude:",
 'Club.']

In [10]:
len(bdata)

2659723

In [11]:
len(edata)

2659723

In [9]:
trainbn = [i for i in bdata[:-532723]]
val_bn = [i for i in bdata[-532723:]]
len(trainbn)

2127000

In [10]:
val_bn[:10]

['নেলী লুথছার',
 'অপরদিকে বৃহদাকার গভীর উপত্যকা',
 'তার বলীখেলার জনপ্রিয়তা এখনও অক্ষুণ্ণ রয়েছে। খেলার আগে ঢোল বাজিয়ে প্রচারকার্য চালানো হয়।',
 'তুমি গিয়ে পো কে সর্তক কর।',
 '৪ হাত = ১ ধনু (৬ ফুট)',
 'ইতিহাস.',
 'অনেকেই হয়তো তাকে চিনে না, তাকে গ্রাহ্য করে না, বা এমনকি তার কথা মনেও করে না, যদিও তিনিই সম্প্রদায়টিকে সরাসরি সাহায্য এবং উন্নতিতে সহায়তা করছেন।',
 'বৈশিষ্ট্য যোগ করা সম্ভব হয়নি',
 'এগারো শতকের ভাস্কর্যসমূহের পশ্চাৎপটের পাথরে লতাপাতা ও নানা উদ্ভিদ বিষয়ক অলঙ্করণ শোভা পায়। মূল মূর্তির গায়ে দেখা যায় অলঙ্কারের বাহুল্য এবং পাথর খোদাই করে সূক্ষ্ম কারুকাজ ফুটিয়ে তোলা হতো।',
 'নির্বোধ']

In [11]:
trainen = [i for i in edata[:-532723]]
val_en = [i for i in edata[-532723:]]
len(trainen)

2127000

In [15]:
val_en[:10]

['Nellie Lutcher as Herself',
 'On the other side of an enormous deep valley',
 'Bali khela is given advance publicity by beating drums.',
 '- You must warn Po.',
 '4 hath = 1 dhanu (6 feet)',
 'History.',
 'Many might not know him, care about him, or even remember him, though he is the one who directly helped and advanced the community.',
 'Cannot add attribute',
 'In the 11th century, however, increasing importance was given to vegetal decoration of the back slab, profuse ornamentation of the main figure, and minute execution of details.',
 'ignorant.']

In [16]:
trainbn[-10:]

['নেপালে নবায়নযোগ্য শক্তি হচ্ছে এমন একটা সেক্টর যা নেপালে ছড়িয়ে পড়ছে।',
 'এটা আমাদের বর্তমান দিনের সমতুল্য পরিপূর্ণতাটিকে আরও ভালভাবে বুঝতে সাহায্য করবে।',
 'ভুলত্রুটি সত্ত্বেও প্রার্থনায় নিবিষ্ট থাকুন',
 'ভালোবাসা ছাড়া, রাগ ছাড়া, দুঃখ ছাড়া... নিশ্বাস নেয়া মাত্র ঘড়ির কাটার শব্দ ।',
 'সংবেদনশীল করা',
 'দেখতে চমৎকার, সার্জেন্ট।',
 '১৬০৫ খ্রিস্টাব্দে প্যারিসে জন্মগ্রহণকারী জে.বি টেভার্নিয়ার তাঁর বাবা গ্যাব্রিয়েলের কাছ থেকে ভূগোলচর্চা ও বিদেশ ভ্রমণের অনুপ্রেরণা লাভ করেন। তাঁর বাবা ছিলেন একজন বণিক ও ভূগোলবিদ।',
 'এন্ডিমো স্টুডিওর কার্টুন, অনুমতিক্রমে প্রকাশিত।',
 '১৬৬০ সালে মুজ্জাম খান (মীর জুমলা) আবার রাজধানী ঢাকায় নিয়ে আসেন।',
 'জাহানারা একদিনের আন্তর্জাতিকে অভিষেক ঘটে ২০১১ সালের ২৬ নভেম্বর আয়ারল্যান্ড মহিলা ক্রিকেট দলের বিরুদ্ধে।']

In [17]:
trainen[-10:]

['Renewable energy in Nepal is a sector that is rapidly developing in Nepal.',
 'This will help us better to understand the parallel fulfillment today.',
 'Persevere in Prayer Despite Shortcomings',
 'without love, without anger, without sorrow... breath is just a clock ticking.',
 'sensitise',
 'Look lively, Sergeant.',
 'Born in Paris in 1605, Tavernier got the inspiration for acquiring knowledge in geography and traveling abroad from his father Gabriel, a merchant and a geographer.',
 'Cartoon by Andimoo Studios, used with permission',
 'In 1660, Muazzam Khan (Mir Jumla) again shifted the capital to Dhaka.',
 "Alam made her ODI career against Ireland women's cricket team on November 26, 2011."]

In [12]:
import os
process_data_path = "/kaggle/working/process_data"
os.makedirs(process_data_path, exist_ok=True)

In [13]:
def write_txt_file(file_path, data, encoding="utf-8"):
    with open(file_path, 'w') as file:
        for key in data:
            if isinstance(key, list):
                key = key[0]
            file.write(key+"\n")

In [14]:
write_txt_file(os.path.join(process_data_path, "bn_data.txt"), trainbn)

In [15]:
write_txt_file(os.path.join(process_data_path,"en_data.txt"), trainen)

In [16]:
def merge_data_write_txt_file(file_path, bn_data, en_data, encoding="utf-8"):
    with open(file_path, 'w') as file:
        for be, en in zip(bn_data, en_data):
            file.write(be+"\t"+en+"\n")

In [17]:
merge_data_write_txt_file(os.path.join(process_data_path, "merge_data.txt"), trainbn, trainen)

In [18]:
model_path = "model"
os.makedirs(model_path, exist_ok = True)

In [19]:
import sentencepiece as spm

def train_tokenizer(text_path="text.txt", model_prefix="model/bn_model"):
    spm.SentencePieceTrainer.train(f'--input={text_path} --model_prefix={model_prefix} --user_defined_symbols=<sep>',min_frequency=2)
    bn_sp = spm.SentencePieceProcessor()
    bn_sp.load(os.path.join(model_path, 'bn_model.model'))

In [20]:
bn_data_path = "/kaggle/working/process_data/bn_data.txt"
en_data_path = "/kaggle/working/process_data/en_data.txt"

In [21]:
train_tokenizer(
    text_path = bn_data_path,
    model_prefix = "model/bn_model"
)

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/kaggle/working/process_data/bn_data.txt --model_prefix=model/bn_model --user_defined_symbols=<sep>
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/process_data/bn_data.txt
  input_format: 
  model_prefix: model/bn_model
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <sep>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_

In [23]:
train_tokenizer(
    text_path = en_data_path,
    model_prefix = "model/en_model"
)

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/kaggle/working/process_data/en_data.txt --model_prefix=model/en_model --user_defined_symbols=<sep>
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/process_data/en_data.txt
  input_format: 
  model_prefix: model/en_model
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <sep>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_

In [24]:
bn_tokenizer = spm.SentencePieceProcessor(model_file='/kaggle/working/model/bn_model.model')
en_tokenizer = spm.SentencePieceProcessor(model_file='/kaggle/working/model/en_model.model')

In [25]:
bn_tokenizer.unk_id=999
en_tokenizer.unk_id=999

In [26]:
en_tokenizer.encode("this is a translator code")

[50, 15, 12, 6870, 2266]

In [27]:
bn_tokenizer.encode("এটি একটি অনুবাদক।")

[129, 39, 1222, 29, 4]

In [28]:
print(bn_tokenizer.encode_as_pieces('এটি একটি অনুবাদক।'))
print(bn_tokenizer.encode_as_ids('এটি একটি অনুবাদক।'))

['▁এটি', '▁একটি', '▁অনুবাদ', 'ক', '।']
[129, 39, 1222, 29, 4]


In [31]:
from torchtext.vocab import vocab
def build_vocab(sentences, tokenizer):
    counter = Counter()
    for sentence in sentences:
        if isinstance(sentence, list):
            sentence = sentence[0]
        counter.update(tokenizer.encode(sentence, out_type=str))
    return vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'], special_first=True)

In [34]:
torchtext.__version__

'0.16.2'

In [32]:
bn_vocab = build_vocab(trainbn, bn_tokenizer)
bn_vocab.set_default_index(999)
en_vocab = build_vocab(trainen, en_tokenizer)
en_vocab.set_default_index(999)

In [42]:
import pickle
# open a file, where you want to store the data
file = open('/kaggle/working/model/bn_vocab.pkl', 'wb')
# dump information to that file
pickle.dump(bn_vocab, file)
file.close()
file = open('/kaggle/working/model/en_vocab.pkl', 'wb')
pickle.dump(en_vocab, file)
file.close()

In [33]:
def data_process(bn, en):
    data = []
    for (raw_bn, raw_en) in zip(bn, en):
        bn_tensor_ = torch.tensor([bn_vocab[token] for token in bn_tokenizer.encode(raw_bn, out_type=str)],dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer.encode(raw_en, out_type=str)],dtype=torch.long)
        data.append((bn_tensor_, en_tensor_))
    return data
train_data = data_process(trainbn, trainen)
val_data = data_process(val_bn, val_en)

In [41]:
torch.save(train_data, '/kaggle/working/model/train_data.pth')
torch.save(val_data, '/kaggle/working/model/val_data.pth')

In [34]:
BATCH_SIZE = 150
PAD_IDX = bn_vocab['<pad>']
BOS_IDX = bn_vocab['<bos>']
EOS_IDX = bn_vocab['<eos>']

def generate_batch(data_batch):
    bn_batch, en_batch = [], []
    for (bn_item, en_item) in data_batch:
        bn_batch.append(torch.cat([torch.tensor([BOS_IDX]), bn_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
    bn_batch = pad_sequence(bn_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    return bn_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True, collate_fn=generate_batch)
val_iter = DataLoader(val_data, batch_size=BATCH_SIZE,shuffle=True, collate_fn=generate_batch)

In [35]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class BanglaTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(BanglaTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(
            d_model=emb_size, 
            nhead=NHEAD,
            dim_feedforward=dim_feedforward
            )
        self.transformer_encoder = TransformerEncoder(
            encoder_layer, 
            num_layers=num_encoder_layers
            )
        decoder_layer = TransformerDecoderLayer(
            d_model=emb_size, 
            nhead=NHEAD,
            dim_feedforward=dim_feedforward
            )
        self.transformer_decoder = TransformerDecoder(
            decoder_layer, 
            num_layers=num_decoder_layers
            )

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [36]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [37]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [38]:
from tqdm import tqdm
SRC_VOCAB_SIZE = len(bn_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 150
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
NUM_EPOCHS = 2


transformer = BanglaTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
#         print("training iter : ", idx)
#     for idx in tqdm(range(len(train_iter))):
#         src, tgt = train_iter[idx]
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [39]:
def evaluate(model, val_iter):
    model.eval()
    losses = 0
    for idx, (src, tgt) in (enumerate(val_iter)):
#         print(idx)
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,
                                  src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:,:]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(val_iter)

In [ ]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    train_loss = train_epoch(transformer, train_iter, optimizer)
#     if epoch % 5 == 0:
    val_loss = evaluate(transformer, val_iter)
    end_time = time.time()
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, val loss : {val_loss:.3f} "
          f"Epoch time = {(end_time - start_time):.3f}s"))

        
    # save model + checkpoint to resume training later
    if (epoch%50==0):
        torch.save({
          'epoch': epoch,
          'model_state_dict': transformer.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': train_loss,
          }, 'model/model_checkpoint.tar')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys,torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
    model.eval()
    tokens = [BOS_IDX] + [src_vocab.get_stoi()[tok] for tok in src_tokenizer.encode(src, out_type=str)]+ [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    p_text = " ".join([tgt_vocab.get_itos()[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")
    pts = " ".join(list(map(lambda x : x , p_text.replace(" ", "").split("▁"))))
    return pts.strip()

In [ ]:
text = "আবার সবাইকে যার যার ইচ্ছামতো চলতে দিলে সমতা রক্ষা করা অসম্ভব হয়ে যায়।"
pre = translate(transformer, text, bn_vocab, en_vocab, bn_tokenizer)
print(f"input : {text}")
print(f"prediction: {pre}")

In [ ]:
# save model + checkpoint to resume training later
torch.save({
  'epoch': "Final",
  'model_state_dict': transformer.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'loss': train_loss,
  }, '/kaggle/working/model/model_checkpoint.pt')

**INFERENCE**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
PATH = "/kaggle/working/model/model_checkpoint.pt"

model = BanglaTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)
model.to(device)
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()
        ys = torch.cat([ys,torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
#     model.eval()
    tokens = [BOS_IDX] + [src_vocab.get_stoi()[tok] for tok in src_tokenizer.encode(src, out_type=str)]+ [EOS_IDX]
    num_tokens = len(tokens)
    src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    p_text = " ".join([tgt_vocab.get_itos()[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")
    pts = " ".join(list(map(lambda x : x , p_text.replace(" ", "").split("▁"))))
    return pts.strip()

In [ ]:
for bn,en in zip(bdata[-10:], edata[-10:]):
    #text = "আমি আবার বিয়ে করেছি।"
    pre = translate(model, bn, bn_vocab, en_vocab, bn_tokenizer)
    print(f"input : {bn}")
    print(f"Ground Truth : {en}")
    print(f"prediction: {pre}")
    print("================================")